In [2]:
from dataset import TensorflowDataset
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
# 

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from dataset import TensorflowDataset

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, Dropout, Dense, Bidirectional, TimeDistributed, BatchNormalization, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, MultiHeadAttention, LayerNormalization


from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [15]:
from models import  TransformerBlock

In [27]:
import tensorflow as tf
import numpy as np
from models import Lstm, Gru, Transformer
from dataset import LABELS
import pandas as pd

class final_model:
    def __init__(self, model_path, batch = 32) -> None:

        # assert model in [Lstm, Gru, Transformer]
        # self.model = model(model_path = f'models\\model_{model.__name__.lower()}_final_version.h5')
        self.model = tf.keras.models.load_model(model_path)
        # self.name = self.model.__name__.lower()
        self.name = 'lstm'

        test = pd.read_pickle('extracted_features\\features_test.pkl')
        dataset = tf.data.Dataset.from_tensor_slices((test, np.zeros(len(test))))
        self.test = dataset

        test_silence = pd.read_pickle('extracted_features\\features_test_silence.pkl')
        dataset = tf.data.Dataset.from_tensor_slices((test_silence, np.zeros(len(test_silence))))
        self.test_silence = dataset

        LABELS_dict = {i:LABELS[i] for i in range(len(LABELS))}
        LABELS_dict[10] = 'unknown'
        self.LABELS_dict = LABELS_dict

        self.predicted_labels = None
        self.silence_labels = None

        self.batch = batch

    def predict_silence(self):
        model_sil = tf.keras.models.load_model('models\\lstm_silence.h5')
        self.silence_labels = np.argmax(model_sil.predict(self.test.batch(self.batch)), axis=1)
    
    def predict_label(self):
        self.predicted_labels = np.argmax(self.model.predict(self.test.batch(self.batch)), axis=1)

    def save_results(self):
        x = pd.concat([pd.DataFrame(self.predicted_labels), pd.DataFrame(self.silence_labels)], axis=1)
        x.columns = ['label', 'silence']
        res = np.where(x['silence'] == 0 , 'silence', x['label'].apply(lambda x: self.LABELS_dict[x]))
        res = pd.DataFrame(res)
        res.columns = ['label_prediction']
        res.to_csv(f'results\\submission_{self.name}.csv', index=False, header=True)

In [33]:
d = final_model('models/gru_mod.h5')

In [34]:
d.predict_label()
d.predict_silence()

 208/4955 [>.............................] - ETA: 1:08

In [30]:
d.save_results()

In [5]:
test = pd.read_pickle('extracted_features\\features_test_silence.pkl')

In [6]:
len(test)

158538

In [13]:
len( pd.read_pickle('features_test.pkl'))
# data = 'features_test.pkl'
# while True:
#     packet = s.recv(4096)
#     if not packet: break
#     data += packet

# data_arr = pickle.loads(data)
# print (data_arr)
# s.close()

158538

In [16]:
from dataset import LABELS

In [18]:
LABELS += ['unknown']

In [21]:
test = pd.read_pickle('features_test.pkl')


In [35]:
model = tf.keras.models.load_model('models/model_gru_final_version.h5')

In [27]:
# model = tf.keras.models.load_model('models/model_transformer_final_version.h5')

In [41]:
import tensorflow as tf



# konwertujemy array na TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices(test[0].reshape((1, 39, 44)))

# ładujemy model
model = tf.keras.models.load_model('models/model_gru_final_version.h5')

# wykonujemy predykcję na danych z TensorFlow Dataset
predictions = model.predict(dataset)


ValueError: in user code:

    File "c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users

In [31]:
model.predict(test)

KeyboardInterrupt: 

In [33]:

class ensamble_model:
    def __init__(self):
        # train = pd.read_pickle('features_test_silence.pkl')
        # y_train = np.array([x[1] for x in train])
        # labels = list(np.unique(y_train))
        # self.labels_silence = labels
        # self.silence_det = TensorflowDataset('features_test_silence.pkl', labels=self.labels_silence).dataset
        
        train = pd.read_pickle('features_test.pkl')
        y_train = np.array([x[1] for x in train])
        labels = list(np.unique(y_train))
        self.labels_label = labels
        self.label_det = TensorflowDataset('features_test.pkl', labels=self.labels_label).dataset
    
        self.silence_model = tf.keras.models.load_model('models\\silence_model.h5')
        self.final_model = 'dupa'#tf.keras.models.load_model('models\\final_model.h5')
    
    def predict_silence(self):
        pred = self.silence_model.predict(self.silence_det)


    def save():
        # otwórz plik csv do zapisu
        with open('predictions.csv', mode='w', newline='') as file:
            # utwórz obiekt writer i ustaw separator na przecinek
            writer = csv.writer(file, delimiter=',')
            
            # zapisz nagłówek
            writer.writerow(['col1', 'col2', 'col3', 'col4', 'prediction'])
            
            # zapisz dane i przewidywania
            for i in range(len(data)):
                row = data[i] + [predictions[i]]
                writer.writerow(row)



In [34]:
model = ensamble_model()

TypeError: unhashable type: 'numpy.ndarray'